In [3]:
import polars as pl
import numpy as np

In [4]:
from buckaroo.polars_buckaroo import PolarsBuckarooWidget
PBW = PolarsBuckarooWidget

In [6]:
str_df = pl.DataFrame({
    'a': ["foo", "foobar", "baz",  None,  None], 
    'b': [    1,        2,     3,  None,  None], 
    'c': [ None,     None,  None, "foo", "baz"],
    'd': np.arange(5),
    'e': [    2,       -5,    30,     1,    40]
})
bw = PBW(str_df)
bw

PolarsBuckarooWidget(buckaroo_options={'sampled': ['random'], 'auto_clean': ['aggressive', 'conservative'], 'p…

In [8]:
bw.ac_obj

In [28]:
from functools import reduce
def or_join(a,b):
    return a|b
def str_filter_df(df, needle):
    str_cols = df.select(pl.col(pl.String)).columns
    clauses = [pl.col(c).str.contains(needle) for c in str_cols]
    orred_clause = reduce(or_join, clauses)
    return df.filter(orred_clause)

In [29]:
str_filter_df(str_df, "baz")

a,b,c,d,e
str,i64,str,i64,i64
"""baz""",3,null,2,30
null,null,"""baz""",4,40


In [30]:
str_filter_df(str_df, "ba")

a,b,c,d,e
str,i64,str,i64,i64
"""foobar""",2,null,1,-5
"""baz""",3,null,2,30
null,null,"""baz""",4,40


In [32]:
str_df.filter(pl.col('e').gt(1), pl.col('e').lt(35))

a,b,c,d,e
str,i64,str,i64,i64
"""foo""",1,null,0,2
"""baz""",3,null,2,30


In [35]:
def range_filter_df(df, col, lower, upper):
    clauses = []
    if lower is not None:
        clauses.append(pl.col(col).gt(lower))
    if upper is not None:
        clauses.append(pl.col(col).lt(upper))
    return df.filter(*clauses)


In [36]:
range_filter_df(str_df, 'e', 0, None)

a,b,c,d,e
str,i64,str,i64,i64
"""foo""",1,null,0,2
"""baz""",3,null,2,30
null,null,"""foo""",3,1
null,null,"""baz""",4,40


In [19]:
from functools import reduce

In [21]:
def or_join(a,b):
    return a|b
reduce(or_join, [pl.col(c).str.contains('foo') for c in str_df.select(pl.col(pl.String)).columns])

<Expr ['[(col("a").str.contains([Strin…'] at 0x11813B460>

In [23]:
from functools import reduce
def or_join(a,b):
    return a|b
def str_filter_df(df, needle):
    str_cols = df.select(pl.col(pl.String)).columns
    clauses = [pl.col(c).str.contains(needle) for c in str_cols]
    orred_clause = reduce(or_join, clauses)
    return df.filter(orred_clause)

In [22]:
str_df.filter(reduce(or_join, [pl.col(c).str.contains('foo') for c in str_df.select(pl.col(pl.String)).columns]))

a,b,c,d
str,i64,str,i64
"""foo""",1,null,0
"""foobar""",2,null,1
null,null,"""foo""",3


In [18]:
r[pl.col(c).str.contains('foo') for c in str_df.select(pl.col(pl.String)).columns]

[<Expr ['col("a").str.contains([String(…'] at 0x1183E4610>,
 <Expr ['col("c").str.contains([String(…'] at 0x1183E5390>]

In [17]:
str_df.filter(*[pl.col(c).str.contains('foo') for c in str_df.select(pl.col(pl.String)).columns])

a,b,c,d
str,i64,str,i64


In [13]:
#str_df.filter(F.col(pl.String))
str_df.filter(pl.col('a').str.contains('foo')|pl.col('c').str.contains('foo'))

a,b,c,d
str,i64,str,i64
"""foo""",1,null,0
"""foobar""",2,null,1
null,null,"""foo""",3


In [37]:
from buckaroo.polars_buckaroo import PolarsBuckarooWidget
PBW = PolarsBuckarooWidget

NameError: name 'np' is not defined